In [29]:
# imports
import os
import io
import tokenize
import numpy as np
import pandas as pd

In [30]:
# paths
data_path =  '/home/ourownstory/Documents/SOL/data/'
# zeppelin_folder = '/home/ourownstory/Documents/SOL/data/Zeppelin/token/'
# os.listdir(data_path)
# os.listdir(zeppelin_folder)

In [34]:
files_dict = {
    'root': [],
    'file_name': [],
}
for root, subdirs, files in os.walk(data_path):
    for file_name in files:
        files_dict['root'].append(root[len(data_path):])
        files_dict['file_name'].append(file_name)
        
files = pd.DataFrame.from_dict(files_dict)
root_list = files['root'].values
root_list = [root.split('/') for root in root_list]
files['class'] = [r.pop(0) for r in root_list]
files['company'] = [r.pop(0) for r in root_list]
files['root'] = ["/".join(r) for r in root_list]
files

,file_name,root,class,company
0,PullPayInterface.sol,,notICO,AceBusters
1,ERC20Basic.sol,,notICO,AceBusters
2,Migrations.sol,,notICO,AceBusters
3,ERC223Basic.sol,,notICO,AceBusters
4,ERC20.sol,,notICO,AceBusters
5,ERC223ReceivingContract.sol,,notICO,AceBusters
6,SafeMath.sol,,notICO,AceBusters
7,PullPayment.sol,satelites,notICO,AceBusters
8,Storage.sol,satelites,notICO,AceBusters
9,Nutz.sol,satelites,notICO,AceBusters
